The below Jupyter Notebook has been set up to help individuals quickly understand the residential property landscape in their municipality and identify both the largest property owners and the largest owners of property with code violations. The hope is that this Notebook can be used to help improve code enforcement and structure plans to find and hold large-scale property owners accountable.

The commented lines are intended to help users navigate the code. With questions, please contact John.

In [1]:
# Import Packages
import pandas as pd
import numpy as np

# Get CSV Files for Analysis. You will need to update the file locations below with 
asmdata = pd.read_csv("/Users/johnobrien/Downloads/Current__2021-2022__Assessment_Roll.csv")
covidata = pd.read_csv("/Users/johnobrien/Downloads/Code_Violations.csv")

/Users/johnobrien/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (82,83) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/johnobrien/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (23,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


After downloading the data files, this section of code will do the bulk of the heavy lifting on data analysis and summarization. The result will be two CSV files, one listing the properties owned by the largest property owners (as identified by common mailing address) the other listing the properties owned by the individuals with the most code violations.

The code was developed using downloaded data from Buffalo, NY's Open Data Portal, found here: https://data.buffalony.gov/. Data for your municipality may contain similar data points in different column names - some updates to the code block will be required in this case.

In [2]:
# This code should standardize the data for the basic reporting assuming common metadata can be drawn 
# from the list below. Unnecessary columns for analysis will be dropped.
subset_df = asmdata[['PROP CLASS DESCRIPTION',
                     'PROPERTY CLASS CODE',
                     'OWNER1',
                     'MAIL3',
                     'MAIL4',
                     'HOUSE NUMBER',
                     'STREET',
                     'TOTAL VALUE',
                     'LAND VALUE',
                     'COUNCIL DISTRICT',
                     'CENSUS TRACT',
                     'NEIGHBORHOOD']].copy()
covi = covidata[['Case Number',
                 'Date',
                 'Status',
                 'Code',
                 'Description',
                 'Address',
                 'Neighborhood',
                 'Prop Class']].copy()

# Fix string types for later analysis. Additional data cleaning may be necessary depending on the quality
# of your dataset; these were just particular problems that I've identified in the past.
subset_df['PROPERTY CLASS CODE'] = subset_df['PROPERTY CLASS CODE'].fillna('0')
subset_df['PROPERTY CLASS CODE'] = subset_df['PROPERTY CLASS CODE'].astype("int")
subset_df['TOTAL VALUE'] = subset_df['TOTAL VALUE'].fillna('0')
subset_df['TOTAL VALUE'] = subset_df['TOTAL VALUE'].astype("int")
subset_df['MAIL3'] = subset_df['MAIL3'].astype("str")
subset_df['TOTAL VALUE'] = subset_df['TOTAL VALUE'].astype("int")
covi['Description'] = covi['Description'].fillna("_blank_")
covi['Description'] = covi['Description'].astype("str")
covi['Address'] = covi['Address'].fillna("_blank_")
covi['Address'] = covi['Address'].astype("str")
covi['Prop Class'] = covi['Prop Class'].fillna("0")
covi['Prop Class'] = covi['Prop Class'].astype("int")

# Modify the below to the equivalent property class codes in your municipality. These are typically standardized
# at the State level
single_unit_housing = 210
two_unit_housing = 220
three_unit_housing = 230
mutliple_structure = 281 # In NY, this refers to plots that might contain an Accessory Dwelling Unit
apartment_building = 411
rowhouses = 482

# This code will be used to set aside the main Residential Property Class Codes in both 
# Assessment and CoVi Datasets.
residential_housing1 = subset_df.loc[subset_df['PROPERTY CLASS CODE']==single_unit_housing]
residential_housing2 = subset_df.loc[subset_df['PROPERTY CLASS CODE']==two_unit_housing]
residential_housing3 = subset_df.loc[subset_df['PROPERTY CLASS CODE']==three_unit_housing]
residential_housing4 = subset_df.loc[subset_df['PROPERTY CLASS CODE']==mutliple_structure]
residential_housing5 = subset_df.loc[subset_df['PROPERTY CLASS CODE']==apartment_building]
residential_housing6 = subset_df.loc[subset_df['PROPERTY CLASS CODE']==rowhouses]
rht = residential_housing1.append([residential_housing2,
                                  residential_housing3,
                                  residential_housing4,
                                  residential_housing5,
                                  residential_housing6])

# The lines below will create additional columns which might be beneficial for analysis. The next two lines ensure
# we have complete Address values - initial viewing of CSV showed numerous blanks, and this will simply help
# standardize the values.
rht['ADDRESS'] = rht['HOUSE NUMBER'].astype("str") + " " + rht['STREET']
rht['ADDRESS'] = rht['ADDRESS'].astype("str")

# This block of code will separate the City and State in the Mail4 column, telling us where property owners are
# primarily located. There is a Pivot Table function defined below to further analyze this.
rht['CITY']=''
rht['STATE']=''
rht['CITY'] = rht['MAIL4'].str.split(',',expand=True)[0]
rht['STATE'] = rht['MAIL4'].str.split(',',expand=True)[1]
rht['CITY'] = rht['CITY'].astype("str")
rht['STATE'] = rht['STATE'].astype("str")

# This block of code will help us to identify whether a building is likely owner or renter occupied as determined
# by having the same physical and assessment mailing address.
rht['LIKELY TENURE']=''
rht.loc[rht['MAIL3'].str[0:6] == rht['ADDRESS'].str[0:6],'LIKELY TENURE'] = 'Likely OO'
rht.loc[rht['MAIL3'].str[0:6] != rht['ADDRESS'].str[0:6],'LIKELY TENURE'] = 'Likely RO'

# This block of code will create a new column that only contains the first few characters of the Mail3. This 
# will help us to identify likely common property owners despite different LLCs and misspelt addresses. The 
# analysis enabled here can be limited to just 6 characters as in the above calculation, but for those unfamiliar 
# with the mechanices of this analysis, 10 character strings are used.
rht['MAIL3_Comparison']=''
rht['MAIL3_Comparison']=rht['MAIL3'].str[0:10]

# This dictionary will help standardize common identified errors in the entry of mailing address locations.
# Edit and add this, as needed, to fit the assessment roll being used; obviously, frequent misspellings of Buffalo
# are not universal
rht_replacement_dict = {
    "BFLO": "BUFFALO",
    "BUFFAL0": "BUFFALO",
    "BUFFLAO": "BUFFALO",
    "0UFFALO": "BUFFALO",
    "BUUFALO": "BUFFALO",
    "BUFFLO":"BUFFALO",
    "BUFFFALO":"BUFFALO",
    "BUFFALONY":"BUFFALO",
    "BUFFALO N":"BUFFALO",
    "BUFFAL":"BUFFALO",
    "BUFFRALO":"BUFFALO",
    "W SENECA": "WEST SENECA",
    "E AMHERST": "EAST AMHERST",
    "N TONAWANDA": "NORTH TONAWANDA",
    "BUFFALO 14213 ": "BUFFALO",
    "LIONS HEAD ONTARIO N0H1W0": "LIONS HEAD",
    "TONWANDA":"TONAWANDA"
}
rht['CITY'] = rht['CITY'].replace(rht_replacement_dict)

# This block will combine the Residential Housing Assessment Data with the Code Violation Data to enable analysis
# of 
covi_rht_merge = pd.merge(covi,rht,left_on='Address',right_on='ADDRESS',how='inner')
covi_rht_merge = covi_rht_merge[['Case Number',
                                'Date',
                                'Status',
                                'Code',
                                'Description',
                                'ADDRESS',
                                'PROP CLASS DESCRIPTION',
                                'PROPERTY CLASS CODE',
                                'OWNER1',
                                'MAIL3',
                                'TOTAL VALUE',
                                'LAND VALUE',
                                'COUNCIL DISTRICT',
                                'CENSUS TRACT',
                                'NEIGHBORHOOD',
                                'CITY',
                                'STATE',
                                'LIKELY TENURE',
                                'MAIL3_Comparison']].copy()

# The following definitions of 'Status' are true for Buffalo's Code Violationa data; this section might need to be
# modified for your municipality:
# 'Complied' means the Violation was abated without housing court intervention.
# 'Closed' means the case was sent to housing court, but the cited issue could still exist.
# 'Active' means that Violations have been opened, but either not checked for compliance or sent to court yet.
# The below code will group Active violations at residential properties by the MAIL3 Comparison column values
covi_active = covi_rht_merge.loc[covi_rht_merge['Status'] == 'ACTIVE']
crm = covi_active.pivot_table(index='MAIL3_Comparison',
                                 values='Case Number',
                                 columns='PROPERTY CLASS CODE',
                                 aggfunc='count',
                                 margins=True,
                                 fill_value=0)
crm = crm.sort_values(by='All',ascending=False)

# This block will sort the properties by comparable property addresses - this is complicated by PO Boxes,
# but workarounds do exist. It will produce a list of the largest property 
likelyRO = rht.loc[rht['LIKELY TENURE'] == 'Likely RO']
owneradd = likelyRO.pivot_table(index=['MAIL3_Comparison'],
                     values='ADDRESS',
                     columns='PROPERTY CLASS CODE',
                     aggfunc='count',
                     margins=True,
                     fill_value=0)
owneradd = owneradd.sort_values(by='All',ascending=False)

# Now that we have a merged table that will allow us to aggregate Code Violations by Property Information,
# we can group and aggregate that information to find some of the larger problematic property owners
crm_subset = []
crm_subset = crm.head(51).index.tolist()
crm_subset = np.delete(crm_subset,[0]) # Below line deletes the 'All' row in the pivot table
# These lines will filter the Residential Housing dataset down to only those properties owned by the 50 largest
# property owners of properties with 
rht_crm_subset = rht.loc[rht['MAIL3_Comparison'].isin(crm_subset)]
rht_crm_subset = rht_crm_subset.sort_values(by=['MAIL3'],ascending=False)

# This information can be compared to the CSV produced below, which simply aggregates the largest individual
# residential property owners
most_props_subset = []
most_props_subset = owneradd.head(51).index.tolist()
most_props_subset = np.delete(most_props_subset,[1])
mps = rht.loc[rht['MAIL3_Comparison'].isin(most_props_subset)]
mps = mps.sort_values(by=['MAIL3'],ascending=False)

# The two lines below will export the information to a CSV - you may want to update the folder that information
# will be directed to
rht_crm_subset.to_csv("all_properties_of_largest_violators.csv",index=False,float_format='%.2f')
mps.to_csv("most_properties.csv",index=False,float_format='%.2f')

At this juncture, we should have a relatively complete set of city-wide data that can be furhter analyzed and summarized through Pivot Tables. 

The below are some code snippets that might help further illustrate the structure of your municipalities residential housing market. To run them, it might be easiest to move (either Cut or Copy to a standalone cell) the snippet. These might be beneficial in building a broader understanding of the nature of a local residential housing landscape.

In [3]:
# This will produce a pivot table summarizing the towns and cities where individual property owners live
#  (or at the very least, where they have their tax bills sent - likely a measure of dwelling or location)
pt = rht.pivot_table(index='CITY',
                     values='ADDRESS',
                     columns='LIKELY TENURE',
                     aggfunc='count',
                     fill_value=0)
pt = pt.sort_values(by=['Likely OO'],ascending=False)
pt.head()

# This block of code will perform a similar analysis as above but include the Property Types in each
pt2 = rht.pivot_table(index='CITY',
                     values='ADDRESS',
                     columns=['LIKELY TENURE','PROPERTY CLASS CODE'],
                     aggfunc='count',
                     margins=True,
                     fill_value=0)
pt2 = pt2.sort_values(by=['All'],ascending=False)
pt2.head()

# This will produce a pivot table identifying the likely tenure of all properties within the City's 
# Planning Neighborhoods and organizes them by the percentage of likely owner-occupied properties.
nt = rht.pivot_table(index='NEIGHBORHOOD',
                     values='ADDRESS',
                     columns='LIKELY TENURE',
                     aggfunc='count',
                     fill_value=0)
nt['% Likely OO'] = nt['Likely OO']/(nt['Likely OO'] + nt['Likely RO'])
nt = nt.sort_values(by='% Likely OO', ascending=False)
nt.head()

# This code will calculate the difference in price between owner-occupied and renter-occupied singles and doubles,
# and what is clear is that in all planning neighborhoods, likely owner-occupied buildings are worth more than their
# likely renter-occupied counterparts.
singdoub = rht.loc[subset_df['PROPERTY CLASS CODE']== 210]
singdoub = singdoub.append(rht.loc[subset_df['PROPERTY CLASS CODE']==220])
nmeanval = singdoub.pivot_table(index='NEIGHBORHOOD',
                                      values='TOTAL VALUE', 
                                      columns='LIKELY TENURE', 
                                      aggfunc='median',
                                      fill_value=0)
nmeanval['% Diff. OO'] = (nmeanval['Likely OO'] - nmeanval['Likely RO']) / nmeanval['Likely OO']
nmeanval = round(nmeanval,2)
nmeanval = nmeanval.sort_values(by='% Diff. OO', ascending=False)
nmeanval.head()

#This set of code will focus solely on Singles and Doubles
owners_singdoub = singdoub.pivot_table(index=['OWNER1','MAIL3_Comparison'],
                     values='ADDRESS',
                     aggfunc='count',
                     fill_value=0)
owners_singdoub = owners_singdoub.sort_values(by='ADDRESS',ascending=False)
owners_singdoub.head()

# This code can be used to identify what property types and neighborhoods a given mailing address owns property in.
# Property owner address can be specified in the first row using the defined Mailing Address Comparison column.
testprint = rht.loc[rht['MAIL3_Comparison']=='91 CAUDLE ']
tp_table = testprint.pivot_table(index=['OWNER1'],
                     values='ADDRESS',
                     columns=['NEIGHBORHOOD','PROPERTY CLASS CODE'],
                     aggfunc='count',
                     margins=True,
                     fill_value=0)
tp_table = tp_table.sort_values(by=['All'],ascending=False)
tp_table

NEIGHBORHOOD                  Black Rock     Broadway Fillmore  \
PROPERTY CLASS CODE                  210 220               210   
OWNER1                                                           
All                                    3   7                 1   
LHBP PROPERTIES LLC                    2   5                 0   
PENINSULA PROPERTY HOLDINGS            1   0                 1   
PENINSULA WHOLESALE HOLDINGS           0   1                 0   
JWAF REAL ESTATE HOLDINGS              0   0                 0   
M&A PROPERTY SOLUTIONS LLC             0   0                 0   
PENINSULA WHOLESALE                    0   0                 0   
PENNINSULA WHOLESALE HOLDINGS          0   1                 0   

NEIGHBORHOOD                  Elmwood Bidwell Elmwood Bryant Fillmore-Leroy  \
PROPERTY CLASS CODE                       411            220            210   
OWNER1                                                                        
All                                         1              1              1   
LHBP PROPERTIES LLC                         0              0              0   
PENINSULA PROPERTY HOLDINGS                 0              1              1   
PENINSULA WHOLESALE HOLDINGS                0              0              0   
JWAF REAL ESTATE HOLDINGS                   0              0              0   
M&A PROPERTY SOLUTIONS LLC                  1              0              0   
PENINSULA WHOLESALE                         0              0              0   
PENNINSULA WHOLESALE HOLDINGS               0              0              0   

NEIGHBORHOOD                      Grant-Amherst          ... Schiller Park  \
PROPERTY CLASS CODE           220           210 220 230  ...           220   
OWNER1                                                   ...                 
All                             1             1   3   1  ...             1   
LHBP PROPERTIES LLC             0             1   1   1  ...             0   
PENINSULA PROPERTY HOLDINGS     1             0   0   0  ...             1   
PENINSULA WHOLESALE HOLDINGS    0             0   1   0  ...             0   
JWAF REAL ESTATE HOLDINGS       0             0   0   0  ...             0   
M&A PROPERTY SOLUTIONS LLC      0             0   0   0  ...             0   
PENINSULA WHOLESALE             0             0   1   0  ...             0   
PENNINSULA WHOLESALE HOLDINGS   0             0   0   0  ...             0   

NEIGHBORHOOD                  Seneca-Cazenovia     South Park  \
PROPERTY CLASS CODE                        210 220        220   
OWNER1                                                          
All                                          1   4          1   
LHBP PROPERTIES LLC                          0   0          0   
PENINSULA PROPERTY HOLDINGS                  0   2          1   
PENINSULA WHOLESALE HOLDINGS                 0   2          0   
JWAF REAL ESTATE HOLDINGS                    0   0          0   
M&A PROPERTY SOLUTIONS LLC                   1   0          0   
PENINSULA WHOLESALE                          0   0          0   
PENNINSULA WHOLESALE HOLDINGS                0   0          0   

NEIGHBORHOOD                  University Heights     Upper West Side  \
PROPERTY CLASS CODE                          210 220             281   
OWNER1                                                                 
All                                            1   7               2   
LHBP PROPERTIES LLC                            0   0               0   
PENINSULA PROPERTY HOLDINGS                    1   7               2   
PENINSULA WHOLESALE HOLDINGS                   0   0               0   
JWAF REAL ESTATE HOLDINGS                      0   0               0   
M&A PROPERTY SOLUTIONS LLC                     0   0               0   
PENINSULA WHOLESALE                            0   0               0   
PENNINSULA WHOLESALE HOLDINGS                  0   0               0   

NEIGHBORHOOD                  West 